In [ ]:

import pandas as pd

# Create a small dataset for water quality
data = {
    "pH": [6.5, 7.2, 5.8, 8.1, 7.0, 6.8],
    "turbidity": [2.5, 1.0, 3.8, 0.5, 2.0, 3.2],
    "dissolved_oxygen": [7.0, 8.2, 5.5, 9.0, 7.5, 6.0],
    "nitrate": [12, 5, 20, 3, 15, 18],
    "temperature": [25, 22, 28, 20, 24, 27],
    "Quality": ["Good", "Good", "Bad", "Good", "Good", "Bad"]
}

df = pd.DataFrame(data)
df.to_csv("dataset.csv", index=False)  # save CSV in Colab environment

print("✅ Dataset created and saved as dataset.csv")
print(df)


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

# ---------------------------
# Load Dataset
# ---------------------------
df = pd.read_csv("dataset.csv")
df = df.drop_duplicates()

# Encode label
df['Quality_label'] = df['Quality'].map({'Good': 1, 'Bad': 0})

# Features & Target
X = df[['pH','turbidity','dissolved_oxygen','nitrate','temperature']]
y = df['Quality_label']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ---------------------------
# Models
# ---------------------------
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(max_depth=5, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"\n{name}")
    print("Accuracy:", acc)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# ---------------------------
# Compare Accuracies
# ---------------------------
plt.bar(results.keys(), results.values(), color=['blue','green','orange'])
plt.title("Model Accuracy Comparison")
plt.ylabel("Accuracy")
plt.show()

# ---------------------------
# Feature Importance (Random Forest)
# ---------------------------
rf_model = models["Random Forest"]
importances = rf_model.feature_importances_
plt.bar(X.columns, importances)
plt.title("Feature Importance (Random Forest)")
plt.show()

# ---------------------------
# ROC Curve (Random Forest)
# ---------------------------
y_prob = rf_model.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y_test, y_prob):.2f}")
plt.plot([0,1],[0,1],'--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Random Forest")
plt.legend()
plt.show()
